<a href="https://colab.research.google.com/github/Fernandomn/busca_em_documentos_com_google_ia/blob/main/busca_em_documentos_com_google_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install -q -U google-generativeai

In [23]:

import numpy as np
import pandas as pd
import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get("SECRET_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [24]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [25]:
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A próxima geralçai de IA para desenvolvedores e Google Workspace"
  "\n"
  "Artigo completo:\n"
  "\n"
  "Gemini API & Google AI Studio: Uma maneira acessível de explrar e criar protótipos com aplicações de IA generativa.")

embeddings = genai.embed_content(model="models/embedding-001", content=sample_text, title= title, task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.049814284, -0.03628954, -0.014236484, 0.02464888, 0.066685915, -0.007486437, -0.03341526, -0.01978834, 0.06617826, 0.05812791, 0.009750153, 0.00990188, -0.04221041, -0.024613405, 0.0066298577, -0.027359124, 0.020819847, -0.013216503, -0.03200663, -0.0042787693, 0.0055521685, 0.010079461, -0.030144837, -0.06559075, -0.016993675, 0.014541894, 0.020134231, -0.035480164, -0.030048244, 0.028638829, -0.048972126, 0.05859957, -0.02961559, 0.007506602, -0.0391494, -0.042127103, -0.022048084, -0.043866634, -0.0038288112, -0.010316808, 0.011899942, -0.084969334, -0.010371667, 0.029304715, -0.008855555, -0.022845648, 0.048686985, 0.039565448, 0.013150206, -0.06199309, 0.03449777, 0.019124683, 0.070125766, -0.04342006, -0.006667396, -0.0033562994, 0.030991713, -0.03715619, 0.025944222, 0.005729336, -0.0036407164, 0.015922256, -0.016682789, 0.05397726, 0.021460038, -0.057672076, -0.03964061, -0.0036458306, 0.014652567, 0.04949586, 0.015993668, 0.016495481, 0.059388917, -0.06156261,

In [26]:
DOCUMENT1 = {
    "title": "Operating the Climate Control System",
    "content": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}
DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}
DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [27]:
df = pd.DataFrame(documents)
df

,title,content
0,Operating the Climate Control System,Your Googlecar has a climate control system th...
1,Touchscreen,Your Googlecar has a large touchscreen display...
2,Shifting Gears,Your Googlecar has an automatic transmission. ...


In [28]:
df.columns = ["Título", "Conteúdo"]
df

,Título,Conteúdo
0,Operating the Climate Control System,Your Googlecar has a climate control system th...
1,Touchscreen,Your Googlecar has a large touchscreen display...
2,Shifting Gears,Your Googlecar has an automatic transmission. ...


In [29]:
model = "models/embedding-001"

In [30]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                             content=text,
                             title=title,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [31]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Título"], row["Conteúdo"]), axis=1)
df

,Título,Conteúdo,Embeddings
0,Operating the Climate Control System,Your Googlecar has a climate control system th...,"[-0.033361107, -0.021217084, -0.049581926, -0...."
1,Touchscreen,Your Googlecar has a large touchscreen display...,"[0.009660736, -0.030662702, -0.017281422, -0.0..."
2,Shifting Gears,Your Googlecar has an automatic transmission. ...,"[-0.04270796, -0.007160868, -0.03242516, -0.02..."


In [32]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                             content=consulta,
                             task_type="RETRIEVAL_QUERY")["embedding"]
  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteúdo"]

In [33]:
consulta = "Como mudar a marcha do Carro do Google?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.


In [34]:
prompt = f"Reescreva esse trecho de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {consulta}"

generation_config = {"candidate_count": 1, "temperature": 0.5}
safety_settings = {
    "HARASSMENT":"BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
    }

model_2 = genai.GenerativeModel("gemini-1.0-pro", generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

Ei, quer saber como trocar a marcha do carango do Google?
